In [2]:
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, Convolution2D, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD
from keras import backend as K
from keras.utils import np_utils
import pandas
import sys
import os
import cv2, numpy as np

Using TensorFlow backend.
C:\Users\Bloris\Anaconda3\envs\bloris\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Bloris\Anaconda3\envs\bloris\lib\site-packages\tensorflow\python\framework\dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Bloris\Anaconda3\envs\bloris\lib\site-packages\tensorflow\python\framework\dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Bloris\Anaconda3\envs\bloris\lib

In [3]:
img_width, img_height = 300, 300
epochs = 20
batch_size = 20

In [4]:
def file_read(filename):
    x,x1,x2=[],[],[]
    f = open(filename+'.txt', 'r')
                
    y=list(map(int, f.readline()[:-1]))

    for i in range(3):
        tmp=list(map(int, f.readline()[:-1].split(' ')[:-1]))
        x1.append([tmp])
    
    f.readline()
    f.readline()
    
    for i in range(3):
        tmp=list(map(int, f.readline()[:-1].split(' ')[:-1]))
        x2.append([tmp])
    
    x1 = list(np.array(x1).T)
    x2 = list(np.array(x2).T)
    
    x=[x1,x2]
    f.close()
    
    return x,y

In [17]:
def load_data(data_path, input_width,input_height):
    print('Loading data... ', end='')
    columns = ['data','label']
    df = pandas.DataFrame(columns=columns)
    x,y=[],[]
    for root, subFolder, files in os.walk(data_path):
        for item in files:
            if item.endswith(".png") or item.endswith(".jpg") :
                fileNamePath = str(os.path.join(root,item))
                im = cv2.imread(fileNamePath)
                im = cv2.resize(im, (input_width,input_height))
                imgArray = np.asarray(im)
                
                classStr = ''.join(fileNamePath.split("\\")[-2:-1])
                xtmp,ytmp=file_read(data_path+"/"+classStr+"/"+item[:-4])
                x.append(xtmp)
                y.append(ytmp)
                if classStr == 'ar':
                    classStr = 0
                else:
                    classStr = 1
                    
                df.loc[len(df)] = [imgArray, classStr]
    data = np.array(df['data'].tolist()) 
    label = np.array(df['label'].tolist())
    print('%d data loaded.'%len(df))
    return data,label,x,y

In [19]:
XC,YC,XH,YH = load_data('data/val', img_width,img_height)
XH,YH=np.array(XH),np.array(YH)
XH = XH.reshape(len(XH),2,256,3)
XH = XH.astype('float32')
XH /= 255
YH = np_utils.to_categorical(YH)

Loading data... 50 data loaded.


In [20]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [21]:
CNN_Model=load_model("weights_cnn_fit_aug.h5")
HNN_Model=load_model("weights_hnn_fit_aug.h5")

CNN_res = CNN_Model.predict_proba(XC)
HNN_res = HNN_Model.predict_proba(XH)

In [22]:
HNN_reshape = HNN_res.T[0].T.reshape(50,1)

In [23]:
Ensemble_res = ((HNN_reshape*(0.86)*(0.86)+(CNN_res*(0.81)*(0.81))/((0.81)*(0.81)+(0.86)*(0.86))))

In [24]:
cnt =0
for i in range(len(Ensemble_res)):
    if Ensemble_res[i] >= 0.5 and YC[i] == 1:
        cnt +=1
    elif Ensemble_res[i] < 0.5 and YC[i] == 0:
        cnt +=1
print(cnt/50)

0.94
